In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import pandas as pd
import json
import time
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string 
import unicodedata as ud
from greek_stemmer import GreekStemmer
import pymongo
import numpy as np
import re
import networkx as nx

In [18]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
database = mongo_client["GreekParliamentProceedings"]
collection = database["Database"]
dataframe = pd.DataFrame(list(collection.find()))

In [52]:
#test, pls ignore
dataframe1 = list(collection.find({ }, { "_id": 1, "speech": 1 })[:100])
for i, row in enumerate(dataframe1):
    #print(row["_id"])
    pass

In [54]:
#test, pls ignore
ticks = [x for x in range(0,1200234,100000)]
ticks.append(1200234)
for i in range(len(ticks)-1):
    #print(ticks[i],ticks[i+1])
    pass

In [1]:
import indexer as ind
import pandas as pd
#dataframe = pd.read_csv('Greek_Parliament_Proceedings_1989_2020.csv')

index, database, tokens, size_distribution = ind.create_index(total_documents=300, chunksize=100)

Length of chunk:  100
CHUNK 1  FINISHED
Length of chunk:  100
CHUNK 2  FINISHED
Length of chunk:  100
CHUNK 3  FINISHED


In [3]:
import indexer as ind
import pandas as pd
dataframe = pd.read_csv('Greek_Parliament_Proceedings_1989_2020.csv')


In [16]:
import pandas as pd
import pymongo
#ind.insert_db(dataframe=dataframe, database_name="GreekParliamentProceedings")

Index(['member_name', 'sitting_date', 'parliamentary_period',
       'parliamentary_session', 'parliamentary_sitting', 'political_party',
       'government', 'member_region', 'roles', 'member_gender', 'speech'],
      dtype='object')


In [ ]:
import pymongo
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
inverted_index = mongo_client["GreekParliamentProceedings"]
collection = inverted_index["InvertedIndex"]
for i in collection.find({"list.postinglist":{"$gt":900000}}):
    print(i["_id"])

In [ ]:
total_docs = set()
for word in tokens.keys():
    for key in tokens[word]["postinglist"].keys():
        total_docs.add(int(key))
print((total_docs))

In [ ]:
dataframe = pd.read_csv('Greek_Parliament_Proceedings_1989_2020.csv', chunksize=10000)
graphs = create_index(dataframe)

In [ ]:
print(len(graphs))

In [ ]:
#print(graphs[0].edges)
import matplotlib.pyplot as plt
 
plt.savefig('graph.png')
# larger figure size
#pos = nx.circular_layout(graphs[2])

plt.figure(3,figsize=(12,12)) 
nx.draw_networkx(graphs[2], with_labels=True,node_size=80,font_size=15)
plt.show()

In [ ]:
print(len(graphs[456].nodes))
#567

In [ ]:
res = nx.k_core(graphs[456])
print(len(res.nodes))
print((res.nodes))

## Networkx library test

In [ ]:
#networkx library
import networkx as nx
g = nx.Graph()
l = ['1','2','3']
#g.add_nodes_from(l)

for word in l:
    if(word not in list(g.nodes)):
        g.add_node(word)
for e in list(g.nodes):
    print(e)
    
g.add_edge('1','2')
g.add_edge('2','3')
if(('1','2') in g.edges):
    print(' in')
for e in list(g.edges):
    print(e)